In [11]:
import geopandas as gpd
from shapely.wkt import loads
from shapely.wkt import dumps

# Assuming you have a WKT string
wkt_string = "LINESTRING (-17 51, 20 40, 30 60)"

# Convert the WKT string to a Shapely geometry object
geometry = loads(wkt_string)
print(dumps(geometry))
print(type(geometry))
# Create a GeoSeries from the Shapely geometry object
geo_series = gpd.GeoSeries(geometry)

# Now you have a GeoSeries containing the geometry
print(geo_series)
print(geo_series[0])


LINESTRING (-17.0000000000000000 51.0000000000000000, 20.0000000000000000 40.0000000000000000, 30.0000000000000000 60.0000000000000000)
<class 'shapely.geometry.linestring.LineString'>
0    LINESTRING (-17.00000 51.00000, 20.00000 40.00...
dtype: geometry
LINESTRING (-17 51, 20 40, 30 60)


In [17]:
from collections import OrderedDict
from flask import jsonify
import json
# Create an ordered dictionary to maintain the order of keys
response_data = OrderedDict([
    ('locationID', 4),
    ('decimalLongitude', 5),
    ('coordinatePrecision', 0.0000001)
])
response_data
# Convert the ordered dictionary to JSON response
json.dumps(response_data)

'{"locationID": 4, "decimalLongitude": 5, "coordinatePrecision": 1e-07}'

In [21]:
# Simple check to assume coordinates are in EPSG4326
def isLatLon(lon, lat):
  ok = True
  if lon > 180 or lon < -180:
    ok = False
  if lat > 90 or lat < -90:
    ok = False
  return ok  

isLatLon(45, 7)

True

In [34]:
from flask import Flask, request, jsonify
# json.dumps(OrderedDict([('Error', "Geometry does not appear to be in EPSG:4326 (Lat/Lon)")]))
# json.dumps({'Error', 'Geometry does not appear to be in EPSG:4326 (Lat/Lon)'})
# error_data = {'Error': "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"}
json.dumps({"Error": "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"})
# Convert the dictionary to a JSON string
# json_response = json.dumps(error_data)
# json_response

'{"Error": "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"}'

In [55]:
from shapely.geometry import Polygon, MultiPolygon, LineString, MultiLineString


def isLatLon(lat, lon): 
  ok = True
  if lon > 180 or lon < -180:
    ok = False
  if lat > 90 or lat < -90:
    ok = False
  return ok  

def iterate_coordinates(geometry):
    if geometry.geom_type == 'Polygon':
        for point in geometry.exterior.coords:
            yield point
    elif geometry.geom_type == 'MultiPolygon':
        for polygon in geometry.geoms:
            for point in polygon.exterior.coords:
                yield point
    elif geometry.geom_type == 'LineString':
        for point in geometry.coords:
            yield point
    elif geometry.geom_type == 'MultiLineString':
        for linestring in geometry.geoms:
            for point in linestring.coords:
                yield point

def wktIsLatLon(wkt):
  wktOK = True
  for point in iterate_coordinates(wkt): 
      if not isLatLon(point[1], point[0]):
        wktOK = False
        break
  return wktOK

# Example geometries
polygon = Polygon([(0, 0), (10, 10), (1, 1), (0, 1), (0, 0)])
multi_polygon = MultiPolygon([polygon])
line_string = LineString([(0, 0), (1, 1), (200, 0)])
multi_line_string = MultiLineString([line_string])

wktIsLatLon(polygon)


All points ok


In [56]:
import geopandas as gpd
s = '{"type": "Feature", "properties": {}, "geometry": {"type": "LineString", "coordinates": [[[-50.29953, 58.385518], [-26.115189, 65.000261], [-44.378586, 41.740066]]]}}'
gpd.read_file(s, driver = 'GeoJSON')

,geometry
0,None


In [57]:
json.dumps(("Error", "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"))

'["Error", "Geometry does not appear to be in EPSG:4326 (Lat/Lon)"]'

In [14]:
import geopandas as gpd
from shapely.geometry import shape

# Define your JSON data
json_data = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "MultiPolygon",
    "coordinates": [
      [
        [
          [0, 0],
          [10, 0],
          [10, 10],
          [0, 10],
          [0, 0]
        ],
        [
          [2, 2],
          [2, 8],
          [8, 8],
          [8, 2],
          [2, 2]
        ]
      ],
      [
        [
          [20, 20],
          [30, 20],
          [30, 30],
          [20, 30],
          [20, 20]
        ],
        [
          [22, 22],
          [22, 28],
          [28, 28],
          [28, 22],
          [22, 22]
        ]
      ]
    ]
  }
}


# Convert JSON data to a GeoDataFrame
geometry = shape(json_data['geometry'])
gdf = gpd.GeoDataFrame(geometry=[geometry])

# Convert GeoDataFrame to GeoSeries
geoseries = gdf['geometry']

print(geoseries[0])

MULTIPOLYGON (((0 0, 10 0, 10 10, 0 10, 0 0), (2 2, 2 8, 8 8, 8 2, 2 2)), ((20 20, 30 20, 30 30, 20 30, 20 20), (22 22, 22 28, 28 28, 28 22, 22 22)))


,x,y
0,0.0,0.0
0,10.0,0.0
0,10.0,10.0
0,0.0,10.0
0,0.0,0.0
0,2.0,2.0
0,2.0,8.0
0,8.0,8.0
0,8.0,2.0
0,2.0,2.0


In [15]:
geoseries.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[0.0, 0.0], [10.0, 0.0], [10.0, 10.0], [0.0, 10.0], [0.0, 0.0]], [[2.0, 2.0], [2.0, 8.0], [8.0, 8.0], [8.0, 2.0], [2.0, 2.0]]], [[[20.0, 20.0], [30.0, 20.0], [30.0, 30.0], [20.0, 30.0], [20.0, 20.0]], [[22.0, 22.0], [22.0, 28.0], [28.0, 28.0], [28.0, 22.0], [22.0, 22.0]]]]}, "bbox": [0.0, 0.0, 30.0, 30.0]}], "bbox": [0.0, 0.0, 30.0, 30.0]}'

In [32]:
coordinates_df = geoseries.get_coordinates()
for index, row in coordinates_df.iterrows():
    # Access row data using row['column_name']
    print(row['x'])

0.0
10.0
10.0
0.0
0.0
2.0
2.0
8.0
8.0
2.0
20.0
30.0
30.0
20.0
20.0
22.0
22.0
28.0
28.0
22.0


In [7]:
import json
json_str = '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[37.68234071574153, 9.037578436602471], [37.36213514975508, 6.442555419019049], [36.81783373349052, 3.8883350955368123], [36.05521366351033, 1.39454908786876], [35.07969561005172, -1.0194812357983347], [33.89668688553037, -3.334663250803833], [32.51200587872496, -5.532105596036247], [30.932370439434383, -7.5931546288929415], [29.16592693237443, -9.499503373027851], [27.222787245890167, -11.233378782293968], [25.11552959519741, -12.777808393282426], [22.859608115612094, -14.116960337337986], [20.473609745126893, -15.236541443586004], [17.979299046450826, -16.124227779417826], [15.401405996363003, -16.77009229064069], [12.767139484348617, -17.166987684448777], [10.105447403954548, -17.310841804617674], [7.4460855088427165, -17.20082904529525], [4.818591569449895, -16.839394688931993], [2.2512792636092924, -16.232127262503504], [-0.2296376327206852, -15.387493363663681], [-2.6007046515667285, -14.31646580034837], [-4.8416214529484805, -13.032086143139793], [-6.935365080691516, -11.549005600251668], [-8.868069745327341, -9.883044180260013], [-10.62871430624281, -8.050799518739135], [-12.208678303598532, -6.069326130492842], [-13.601225811726895, -3.9558955158448312], [-14.800967529626512, -1.7278390241646238], [-15.803339565568228, 0.5975306805215617], [-16.60412564895351, 3.002927337470417], [-17.19904015554558, 5.471037450389872], [-17.583383424135246, 7.984415130778189], [-17.751778678676402, 10.525335036685053], [-17.698001322952262, 13.075612830229385], [-17.414916093855478, 15.616402999456653], [-16.894545004270864, 18.127985450498233], [-16.128298282560323, 20.589555457591814], [-15.107409989869868, 22.979036763465817], [-13.823626707246772, 25.272945041787917], [-12.270196653528913, 27.44633827237952], [-10.443190389549175, 29.47290060632094], [-8.343143571554865, 31.32521425422838], [-5.976938658094976, 32.97527504515794], [-3.359734896405657, 34.39529494852405], [-0.5166298967468778, 35.55880219834751], [2.5163675877429252, 36.44199352416266], [5.692491839657627, 37.02521931959415], [8.955809132969517, 37.29441050382834], [12.24425508484867, 37.24221515828536], [15.493746697876865, 36.86863174086518], [18.642664224534407, 36.18101180594347], [21.635876712968493, 35.19343570208819], [24.42763435237112, 33.925593529084864], [26.982996223690282, 32.4013856197292], [29.277845594075746, 30.64747132869302], [31.297823158419362, 28.691951917594462], [33.0366210474436, 26.563301305870603], [34.49405010965715, 24.28958644921076], [35.67418422104612, 21.89796507923057], [36.583760010538874, 19.414417572292525], [37.230906267723796, 16.863658291041855], [37.62420769200715, 14.2691730841742], [37.7720701738396, 11.653337331303707], [37.68234071574153, 9.037578436602471]]]}, "bbox": [-17.751778678676402, -17.310841804617674, 37.7720701738396, 37.29441050382834]}], "bbox": [-17.751778678676402, -17.310841804617674, 37.7720701738396, 37.29441050382834]}'
# json_api = json.loads('{"sec_representation": [{"geometry": {"coordinates": [[[-11.075346,44.169208],[-12.420349,42.265542]]],"type": "Polygon"},      "properties": {},"type": "Feature"}]}')
json_api = json.loads(json_str)
coordinates = json_api["features"][0]["geometry"]["coordinates"]
json_obj = {}
json_obj['sec_representation'] = {}
json_obj['sec_representation']['geometry'] = {}
json_obj['sec_representation']['geometry']['coordinates'] = json.dumps(coordinates)
json_obj['sec_representation']['geometry']['type'] = "Polygon"
json_obj['sec_representation']['properties'] = {}
json_obj['sec_representation']['type'] = "Feature"
json_obj_str = json.dumps(json_obj)
json_obj_str


TypeError: list indices must be integers or slices, not str